In [1]:
import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import numpy as np
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, TrainingArguments, Trainer
from torch.utils.data import Dataset
import logging

from datasets import load_dataset

from datasets import load_dataset

# URL of the TSV file
url = "https://raw.githubusercontent.com/MMU-TDMLab/CompLex/master/train/lcp_single_train.tsv"
test_url = "https://raw.githubusercontent.com/MMU-TDMLab/CompLex/refs/heads/master/test-labels/lcp_single_test.tsv"
# Load the TSV file using the csv format
train_data = load_dataset(
    "csv",
    data_files=url,
    delimiter="\t"  # Specify tab-separated values
)

# Inspect the dataset
print(train_data)


val_data = load_dataset(
    "csv",
    data_files=test_url,
    delimiter="\t"  # Specify tab-separated values
)

# Inspect the dataset
print(val_data)

/home/ubuntu/miniconda3/envs/emnlp_2/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


DatasetDict({
    train: Dataset({
        features: ['id', 'corpus', 'sentence', 'token', 'complexity'],
        num_rows: 7232
    })
})
DatasetDict({
    train: Dataset({
        features: ['id', 'corpus', 'sentence', 'token', 'complexity'],
        num_rows: 887
    })
})


In [2]:
from transformers import AutoTokenizer, AutoModelForMaskedLM, AutoConfig
#from roberta import RobertaForSequenceClassification


model_name = "answerdotai/ModernBERT-large"

#config.num_labels=2
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.padding_side = 'left'

In [3]:
from datasets import DatasetDict

mask_token = tokenizer.mask_token

def generate_prompt(data_point):
    """
    Generates a prompt for evaluating the humor intensity of an edited headline.
    Args:
        data_point (dict): A dictionary containing 'original', 'edit', and 'meanGrade'.
    Returns:
        str: The formatted prompt as a string.
    """
    return f"""# Sentence: {data_point['sentence']} # Word: {data_point['token']} # Output: The complexity score between word and output is"""  # noqa: E501


# Assuming `dataset` is your DatasetDict
def add_label_column(example):

    example['labels'] = float(example['complexity'])
  
    example['input'] = generate_prompt(example)

    
    return example

train_data = train_data['train'].map(add_label_column)
val_data = val_data['train'].map(add_label_column)

In [4]:
from transformers import AutoTokenizer, DataCollatorWithPadding


tokenizer.padding_side = 'left'


# col_to_delete = ['idx']
col_to_delete = ['id', 'corpus', 'sentence', 'token', 'complexity']

mask_token = tokenizer.mask_token
def preprocessing_function(examples):
   
    return tokenizer(examples['input'], truncation=True, max_length=512)

tokenized_train_data = train_data.map(preprocessing_function, batched=True, remove_columns=col_to_delete)
tokenized_val_data = val_data.map(preprocessing_function, batched=True, remove_columns=col_to_delete)
# llama_tokenized_datasets = llama_tokenized_datasets.rename_column("target", "label")
tokenized_train_data.set_format("torch")
tokenized_val_data.set_format("torch")

# Data collator for padding a batch of examples to the maximum length seen in the batch
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Map: 100%|██████████| 887/887 [00:00<00:00, 16264.45 examples/s]


In [5]:
tokenizer.decode(tokenized_train_data['input_ids'][10])


'[CLS]# Sentence: Seven days you shall eat unleavened bread, as I commanded you, at the time appointed in the month Abib; for in the month Abib you came out from Egypt. # Word: days # Output: The complexity score between word and output is[SEP]'

In [6]:
val_data

Dataset({
    features: ['id', 'corpus', 'sentence', 'token', 'complexity', 'labels', 'input'],
    num_rows: 887
})

In [7]:
all_lengths = [len(ids) for ids in tokenized_train_data['input_ids']]
mx = max(all_lengths)


In [8]:
count = sum(len(ids) > 512 for ids in tokenized_train_data['input_ids'])
print(count)


0


In [9]:
import torch
import torch.nn as nn
# from transformers import RobertaForSequenceClassification
from transformers import AutoModelForSequenceClassification
from transformers.activations import ACT2FN
import random
# from modeling import MLMSequenceClassification

config = AutoConfig.from_pretrained(model_name)
config.mask_token_id = tokenizer.mask_token_id
config.num_labels = 1

model = AutoModelForSequenceClassification.from_pretrained(model_name, config=config)

Some weights of ModernBertForSequenceClassification were not initialized from the model checkpoint at answerdotai/ModernBERT-large and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
model

ModernBertForSequenceClassification(
  (model): ModernBertModel(
    (embeddings): ModernBertEmbeddings(
      (tok_embeddings): Embedding(50368, 1024, padding_idx=50283)
      (norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (drop): Dropout(p=0.0, inplace=False)
    )
    (layers): ModuleList(
      (0): ModernBertEncoderLayer(
        (attn_norm): Identity()
        (attn): ModernBertAttention(
          (Wqkv): Linear(in_features=1024, out_features=3072, bias=False)
          (rotary_emb): ModernBertRotaryEmbedding()
          (Wo): Linear(in_features=1024, out_features=1024, bias=False)
          (out_drop): Identity()
        )
        (mlp_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (mlp): ModernBertMLP(
          (Wi): Linear(in_features=1024, out_features=5248, bias=False)
          (act): GELUActivation()
          (drop): Dropout(p=0.0, inplace=False)
          (Wo): Linear(in_features=2624, out_features=1024, bias=False)
        

In [11]:
import RoCoFT

RoCoFT.PEFT(model, method='column', rank=3) 
#targets=['key', 'value', 'dense', 'query'])

In [12]:
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from scipy.stats import pearsonr, spearmanr

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    # If predictions are logits or have extra dimensions, squeeze
    if predictions.ndim > 1:
        predictions = predictions.squeeze()

    mae = mean_absolute_error(labels, predictions)
    mse = mean_squared_error(labels, predictions)
    rmse = np.sqrt(mse)
    r2 = r2_score(labels, predictions)
    
    # Define an "accuracy" for regression:
    # Example: within some threshold tolerance
    tolerance = 0.1  # you can change this
    acc = np.mean(np.abs(predictions - labels) < tolerance)

    pearson_corr, _ = pearsonr(predictions, labels)
    spearman_corr, _ = spearmanr(predictions, labels)

    return {
        "MAE": mae,
        "MSE": mse,
        "RMSE": rmse,
        "Accuracy": acc,
        "R2": r2,
        "Pearson": pearson_corr,
        "Spearman's Rank": spearman_corr
    }


In [13]:
from transformers import TrainingArguments, Trainer

import time
from transformers import Trainer, TrainingArguments
training_args = TrainingArguments(
    output_dir='dir',
    learning_rate=2e-3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=10,
    weight_decay=0.20,
    eval_strategy="steps",
    save_strategy="steps",
    save_total_limit=2,
    save_steps=10000000,
    logging_steps=100,
   
    load_best_model_at_end=True,
    lr_scheduler_type="cosine",  # You can choose from 'linear', 'cosine', 'cosine_with_restarts', 'polynomial', etc.
    warmup_steps=100,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_data,
    eval_dataset=tokenized_val_data,

    data_collator=data_collator,
    compute_metrics=compute_metrics
)

In [14]:
trainer.train()

Step,Training Loss,Validation Loss,Mae,Mse,Rmse,Accuracy,R2,Pearson,Spearman's rank
100,0.080700,0.039615,0.153020,0.039615,0.199035,0.422773,-1.415869,0.032912,0.060721
200,0.027000,0.030097,0.131091,0.030097,0.173485,0.490417,-0.835435,0.168531,0.195623
300,0.021500,0.018772,0.103316,0.018772,0.137012,0.598647,-0.144817,0.287076,0.294931
400,0.016600,0.019089,0.113286,0.019089,0.138162,0.496054,-0.164106,0.353253,0.363109
500,0.018400,0.021390,0.112588,0.021390,0.146253,0.568207,-0.304451,0.363001,0.373049
600,0.019000,0.016943,0.102881,0.016943,0.130166,0.554679,-0.033264,0.308338,0.313344
700,0.017400,0.015289,0.095439,0.015289,0.123650,0.607666,0.067601,0.429489,0.431174
800,0.014200,0.013639,0.089465,0.013639,0.116785,0.635851,0.168261,0.482005,0.460542
900,0.013000,0.014425,0.097858,0.014425,0.120104,0.570462,0.120308,0.519118,0.512199
1000,0.011600,0.014198,0.096695,0.014198,0.119155,0.582864,0.134154,0.575721,0.556195


TrainOutput(global_step=9040, training_loss=0.007192725414591553, metrics={'train_runtime': 3874.6655, 'train_samples_per_second': 18.665, 'train_steps_per_second': 2.333, 'total_flos': 43794279204480.0, 'train_loss': 0.007192725414591553, 'epoch': 10.0})